In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
from torch import nn
import torch.optim as opt
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import glob
import matplotlib.patches as patches
import math
import imageio
#Import Libraries

In [ ]:
#The model
class AttentionBlock(nn.Module):
    def __init__(self, in_features_l, in_features_g, attn_features, up_factor, normalize_attn=True):
        super(AttentionBlock, self).__init__()
        self.up_factor = up_factor
        self.normalize_attn = normalize_attn
        self.W_l = nn.Conv2d(in_channels=in_features_l, out_channels=attn_features, kernel_size=1, padding=0, bias=False)
        self.W_g = nn.Conv2d(in_channels=in_features_g, out_channels=attn_features, kernel_size=1, padding=0, bias=False)
        self.phi = nn.Conv2d(in_channels=attn_features, out_channels=1, kernel_size=1, padding=0, bias=True)
        self.relu = nn.SiLU()
    def forward(self, l, g):
        #print(in_features_g)
        N, C, W, H = l.shape
        l_ = self.W_l(l)
        g_ = self.W_g(g)
        #print(g_.shape)
        c = self.phi(self.relu(l_ + g_)) # batch_sizex1xWxH
        #print(c.shape)
        # compute attn map
        if self.normalize_attn:
            a = (c.view(N,1,-1)).view(N,1,W,H)
        else:
            a = torch.sigmoid(c)
        # re-weight the local feature
        f = torch.mul(a.expand_as(l), l) # batch_sizexCxWxH
        if self.normalize_attn:
            output = f.view(N,C,-1) # weighted sum
        else:
            output = nn.AdaptiveAvgPool2d(f, (1,1)).view(N,C) # global average pooling
        return a, output
    
class Skynet1_4(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv21 = nn.Conv2d(8,128,1,stride = 1)
        self.conv22 = nn.Conv2d(128, 160, 1,stride = 1)
        self.conv23 = nn.Conv2d(160,128, 1,stride = 1)
        self.poolmax2 = nn.MaxPool2d(1,5)
        self.poolavg2 = nn.AvgPool2d(1,5)

        self.poolmax1 = nn.MaxPool1d(1,5)
        self.poolavg1 = nn.AvgPool1d(1,5)
        self.batch1 = nn.BatchNorm1d(2)

        self.conv11 = nn.Conv1d(2,128,1)
        self.batch2 = nn.BatchNorm1d(128)
        self.conv12 = nn.Conv1d(128,160,1)
        self.batch3 = nn.BatchNorm1d(160)
        self.conv13 = nn.Conv1d(160,96,1)
        self.batch4 = nn.BatchNorm1d(96)

        self.attentionmech = AttentionBlock(8, 128, 256, 4, normalize_attn=True)
        self.fc1 = nn.Linear(1920,512)
        self.fc2 = nn.Linear(512,256)
        self.batch6 = nn.BatchNorm1d(512)
        self.batch5 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256,1)
        #self.squeeze = torch.squeeze(1)
        self.relu = nn.SiLU()

    def forward(self,x):
        temp = x
        x = self.relu(self.conv21(x))
        x = self.relu(self.conv22(x))
        x = self.relu(self.conv23(x))
        attention, x = self.attentionmech(temp,x)
        xmax = self.poolmax2(x)
        xavg = self.poolavg2(x)
        x = 0.3*xmax + 0.7*xavg
        #print(x.shape)
        
        #x = torch.flatten(x,start_dim=2,end_dim=3)
        x = self.batch1(x)
        x = self.relu(self.conv11(x))
        x = self.batch2(x)
        x = self.relu(self.conv12(x))
        x = self.batch3(x)
        x = self.relu(self.conv13(x))
        x = self.batch4(x)
        xmax = self.poolmax1(x)
        xavg = self.poolavg1(x)
        x = 0.3*xmax + 0.7*xavg
        x = torch.flatten(x,start_dim=1,end_dim=2)
        #print(x.shape)

        x = self.relu(self.fc1(x))
        #print(x.mT.shape)
        x = self.batch6(x)
        x = self.relu(self.fc2(x))
        #x = self.batch5(x.T)
        x = self.fc3(x)
        return x, attention

In [ ]:
def reformat(frame):
    ballcarrier = 0
    temp = torch.zeros(8,22,22)
    for i in range(len(frame['x'])):
        if frame.iloc[i,'is_on_offence']:
            teamvec1 = frame['is_on_offence']
            teamvec2 = frame['is_on_defence']
        else:
            teamvec2 = frame['is_on_offence']
            teamvec1 = frame['is_on_defence']
        if frame.iloc[i,"is_ballcarrier"]:
            ballcarrier = i
        x1 = frame.iloc[i,'x']
        y1 = frame.iloc[i,'y']
        s1 = frame.iloc[i,'s']
        o1 = frame.iloc[i,'o']
        dir1 = frame.iloc[i,'dir']
        temp[0,0:-2,j] = torch.tensor((frame['x'] - x1).values)
        temp[1,0:-2,j] = torch.tensor((frame['y'] - y1).values)
        temp[2,0:-2,j] = torch.tensor((frame['s'] - s1).values)
        temp[3,0:-2,j] = torch.tensor((frame['dir'] - dir1).values)
        temp[3,0:-2,j] = temp[3,0:-2,j] - (temp[3,0:-2,j] > 360)*-360
        temp[3,0:-2,j] = temp[3,0:-2,j] + (temp[3,0:-2,j] < 0)*-360
        temp[4,0:-2,j] = torch.tensor((np.arctan2((frame['y'] - y1).to_numpy(), -1*(frame['x'] - x1).to_numpy())*180/np.pi))
        temp[4,0:-2,j] = temp[4,0:-2,j] + (1*(-1*(frame['x'] - x1) > 0).values & (((frame['y'] - y1) < 0)*180).values) + (-1*(-1*(frame['x'] - x1) > 0).values & ((frame['y'] - y1) < 0).values)*180
        temp[4,0:-2,j] = temp[4,0:-2,j] -360*(temp[4,0:-2,j] > 360)
        temp[4,0:-2,j] = temp[4,0:-2,j] +360*(temp[4,0:-2,j] < 0)
        temp[4,0:-2,j] -= o1
        temp[5,0:-2,j] = torch.tensor((teamvec2*(((frame['x'] - x1)**2 + (frame['y'] - y1)**2)**0.5) - (~teamvec2)*(((frame['x'] - x1)**2 + (frame['y'] - y1)**2)**0.5)).values)
        #temp2 = torch.zeros((2,22))
        if y1 >= 53.3/2:
            temp[:,i] = 53.3 - y1
        else:
            temp[:,i] = y1
                                    
        temp[-1,i] = 105 - x1
    inds = temp[5,i,:].sort()[1]
    temp = temp[:,:,inds]
    temp = temp[:,inds,:]
    return temp

def predict(frame,model):
    yards_predict, att = model.forward(reformat(frame))
    return yards_predict
    

In [ ]:
#Load the model, requires load file to be in same directory
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Skynet1_4().to(device).eval()
model.load_state_dict(torch.load("modelSkynet15-4fixed.pt"))